In [17]:
import os
import dill
import ipywidgets as widgets
from aepsych.plotting import plot_strat
from aepsych.server import AEPsychServer
from IPython.display import HTML, display
from ipywidgets import Label, Layout, Text, fixed, interact, interact_manual, interactive

# database_path = "/Users/ecortez/Work/Notes/webapp-proj/jupyter_dashboard/data_collection_analysis_tutorial.db"
# This should run whenever a new db is uploaded
# database_path = "/home/ecortez/persistent/private-90d/data_collection_analysis_tutorial.db"
# serv = AEPsychServer(database_path=database_path)
# strat = serv.get_strat_from_replay()

database_path = None
server = AEPsychServer()
strat = None 
# if database_path != None:
#     strat = server.get_strat_from_replay()

# ---------- Style ----------
display(HTML('<h1 style="text-align: center;">Interactive AEPsych</h1>'))
input_style = {"description_width": "initial", "padding": "10px"}
btn_style = Layout(margin="20px 10px")

btn_box_layout = Layout(display="flex", justify_content="flex-end")
file_output = widgets.Output(
    #     layout={"border": "1px solid black", "margin": "10px", "overflow": "scroll", "height": "40px", "padding": "5px"}
)
# ---------- Inputs -----------


def on_value_change(change):
    file_output.clear_output()
    with file_output:
        print(change["new"])


uploader = widgets.FileUpload(description="Resume Session", accept=".db", multiple=False)

input_zero = widgets.Dropdown(
    options=[("Yes Trial", "Detected Trial"), ("No Trial", "Undetected Trial")],
    value="Detected Trial",
    description="0:",
    style=input_style,
)
input_one = widgets.Dropdown(
    options=[("No Trial", "Undetected Trial"), ("Yes Trial", "Detected Trial")],
    value="Undetected Trial",
    description="1:",
    style=input_style,
)

target_level = widgets.BoundedFloatText(
    value=0.75, min=0, max=1, step=0.1, description="target_level:", disabled=False, style=input_style
)

cred_level = widgets.BoundedFloatText(
    value=0.95, min=0, max=1, step=0.1, description="cred_level:", disabled=False, style=input_style
)

x_axis = Text(
    value="Angle (degrees)", placeholder="x axis label", description="x_axis:", disabled=False, style=input_style
)

y_axis = Text(
    value="Detection Probability", placeholder="y axis label", description="y_axis:", disabled=False, style=input_style
)

# Observes input changes
uploader.observe(on_value_change, names="value")
# display(uploader, file_output)

inputs_output = widgets.Output(layout={"border": "1px solid black", "padding": "20px", "margin": "20px"})
input_zero.observe(on_value_change, names="value")
input_one.observe(on_value_change, names="value")
target_level.observe(on_value_change, names="value")
cred_level.observe(on_value_change, names="value")
x_axis.observe(on_value_change, names="value")
y_axis.observe(on_value_change, names="value")
# ------------ Buttons ------------

button_upload = widgets.Button(
    description="Upload", disabled=False, button_style="warning", tooltip="Click to Upload", layout=btn_style
)
button_submit = widgets.Button(
    description="Submit", disabled=False, button_style="warning", tooltip="submit", layout=btn_style
)
button_reset = widgets.Button(
    description="Reset", disabled=False, button_style="danger", tooltip="reset", layout=btn_style
)

# --------- Accordion -----------
accordion = widgets.Accordion(
    children=[
        widgets.VBox([input_zero, input_one]),
        widgets.VBox(
            [
                target_level,
                cred_level,
            ]
        ),
        widgets.VBox([x_axis, y_axis]),
    ]
)
accordion.set_title(0, "Outcome Labels")
accordion.set_title(1, "Parameters")
accordion.set_title(2, "Axis Labels")

# ------------ Tabs -------------
tab = widgets.Tab()

children = [
    widgets.VBox(
        [
            Label("To resume an experiment upload a .db file from a previous session."),
            uploader,
            widgets.Box([button_upload], layout=btn_box_layout),
            file_output,
        ]
    ),
    widgets.VBox(
        [
            accordion,
            widgets.Box([button_submit, button_reset], layout=btn_box_layout),
            Label("Plot output: "),
            inputs_output,
        ]
    ),
]
tab.children = children
tab.set_title(0, "Upload")
tab.set_title(1, "Plot")
display(tab)

def start_server():
    global strat 
    global server
    with file_output: 
        # hard coded db path will be replaced with `database_path`
        server = AEPsychServer(database_path=database_path)
        print("PATH:", database_path)
        # server = AEPsychServer(database_path="/Users/ecortez/Work/voila/databases/data_collection_analysis_tutorial.db")
        try:
            print("Connected...") 
            strat = server.get_strat_from_replay() 
        except:
            print("Unable to connect to server...")  

def display_plot():
    global strat
    inputs_output.clear_output()
    if strat != None: 
        plot_strat(
            strat,
            xlabel=x_axis.value,
            ylabel=y_axis.value,
            yes_label=input_one.value,
            no_label=input_zero.value,
            cred_level=cred_level.value,
            target_level=target_level.value,
        )
    else:
        print("Server has no experiment records!")


def upload():
    global server
    global strat
    global database_path 
    with file_output:
        print("\n -----Now this is how your file looks like:----- \n")
        file_output.clear_output()
        if uploader.value == {}:
            print("No file uploaded")
        else:
            # Maybe add a function to check file format
            for file_name, csv in uploader.value.items():
                current_path = os.getcwd()
                internal_target_path = current_path + "/persistent/private-90d"
                external_target_path = current_path + "/databases"
                if os.path.isdir(internal_target_path):
                    completeName = os.path.join(internal_target_path, file_name)

                elif os.path.isdir(external_target_path):
                    completeName = os.path.join(external_target_path, file_name)
                else:
                    print("File upload unsuccessful")
                serialize_file = dill.dumps(uploader.value)

                with open(completeName, "wb") as f:
                    f.write(serialize_file)
                    database_path = completeName
                    start_server()
                    display_plot()
                    f.close()

                # Need to deserialize in AEPSych server
                # if type(serialize_file) != "dict":
                #     deserialize = dill.loads(serialize_file)
                    # print("OUT", type(deserialize), deserialize == uploader.value)


def submit():
    with inputs_output:
        display_plot()
        file_output.clear_output()
     


def reset():
    with inputs_output:
        inputs_output.clear_output()
        #         plotting options
        x_axis.value = "Angle (degrees)"
        y_axis.value = "Detection Probability"
        input_one.value = "Detected Trial"
        input_zero.value = "Undetected Trial"
        cred_level.value = 0.75
        target_level.value = 0.95
        file_output.clear_output()


# Onclick handler
def upload_clicked(b):
    upload()


def submit_clicked(b):
    submit()


def reset_clicked(b):
    reset()


button_upload.on_click(upload_clicked)
button_submit.on_click(submit_clicked)
button_reset.on_click(reset_clicked)

2022-07-22 12:16:40,276 [INFO   ] Found DB at ./databases/default.db, appending!
